In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import glob
from datetime import date
from datetime import datetime
import math
import requests

In [ ]:
'''from google.colab import drive
drive.mount('/content/gdrive/')'''

In [ ]:
#path = r'/content/gdrive/My Drive/EDA NY/csv/2019'
#path = r'C:\Users\Jacobo\Desktop\citibike2019'
#all_csv = glob.glob(path + "/*.csv")
#li = []
#for filename in all_csv:
#    df = pd.read_csv(filename, index_col=None, header=0)
#    li.append(df)
#df_bike = pd.concat(li, axis=0, ignore_index=True)
month = '12'
df_bike = pd.read_csv('gdrive/My Drive/EDA NY/csv/2019/2019'+month+'-citibike-tripdata.csv')
df_bike.head()

In [ ]:
df_bike = df_bike[['tripduration', 'starttime',  'start station id','start station latitude',
       'start station longitude', 'end station id',
       'end station latitude', 'end station longitude']]
df_bike.head()

In [ ]:
def format_date(date):
  date = date.split('.')[0]
  return date[0:14] + '00:00'
df_bike['starttime'] = df_bike.starttime.apply(lambda x: format_date(x))
df_bike.head()

In [ ]:
df_bike.shape

In [ ]:
df_bike['end station id'].value_counts()

In [ ]:
def join_coords(lat, lon):
  return '{' + f'{lat}, {lon}' + '}'
df_bike['startcoords'] = df_bike.apply(lambda row: join_coords(row["start station latitude"], row["start station longitude"]), axis = 1)
df_bike.head()

In [ ]:
df_bike.startcoords.value_counts()

In [ ]:
unique_coords = df_bike.startcoords.value_counts().rename_axis('coords').reset_index(name='counts')
unique_coords.head()

In [ ]:
unique_coords.shape

In [ ]:
#AQ
df_aq = pd.read_csv('gdrive/My Drive/EDA NY/csv/aqny2019.csv')
df_aq = df_aq[['date', 'parameter', 'location', 'value', 'unit', 'coordinates']]
df_aq.head()

In [ ]:
def get_date(data):
  data = data.replace('}', '').replace('{', '')
  date = data.split(',')[1].split('=')[1].split('-05')[0].replace('T', ' ')
  return date
  #starttime = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
  #return starttime
df_aq['date'] = df_aq.date.apply(lambda x: get_date(x))
df_aq.head()

In [ ]:
df_aq.parameter.value_counts()

In [ ]:
def get_lat(coords):
  coords = coords.replace('{', '').replace('}', '')
  return coords.split(',')[0].split('=')[1]
def get_lon(coords):
  coords = coords.replace('{', '').replace('}', '')
  return coords.split(',')[1].split('=')[1]
df_aq['lat'] = df_aq.coordinates.apply(lambda x: get_lat(x))
df_aq['lon'] = df_aq.coordinates.apply(lambda x: get_lon(x))
df_aq.head()

In [ ]:
unique_aq = df_aq.coordinates.value_counts().rename_axis('coords-aq').reset_index(name='counts')
unique_aq.columns = ['coords', 'count']
unique_aq.head()

In [ ]:
def get_station_name(coords):
  locations = df_aq[df_aq['coordinates'] == coords].location
  if(len(locations) > 0):
    return locations.tolist()[0]
  else:
    return 'NO'
unique_aq['aq_station_name'] = unique_aq.coords.apply(lambda x: get_station_name(x))
unique_aq = unique_aq[['coords', 'aq_station_name']]
unique_aq.head()
#x='{latitude=40.72099, longitude=-74.192894}'
#get_station_name(x)
#df_aq[df_aq['coordinates']==x].location.tolist()[0]

In [ ]:
from math import sin, cos, sqrt, atan2, radians

def get_distance_map_api(lat1,long1, lat2, long2):
  #key from https://developer.mapquest.com/documentation/samples/directions/v2/route/
  r = requests.get('https://www.mapquestapi.com/directions/v2/route?key=lYrP4vF3Uk5zgTiGGuEzQGwGIVDGuy24&unit=k&from='+str(lat1)+'%2C'+str(long1)+'&to='+str(lat2)+'%2C'+str(long2)+'&outFormat=json&ambiguities=ignore&routeType=bicycle&doReverseGeocode=false&enhancedNarrative=false&avoidTimedConditions=true')
  return r.json()['route']['distance']

def get_distance(lat1,long1, lat2, long2):
  # approximate radius of earth in km
  EARTH_RADIUS = 6373.0
  lat1 = radians(float(lat1))
  lon1 = radians(float(long1))
  lat2 = radians(float(lat2))
  lon2 = radians(float(long2))
  dlon = lon2 - lon1
  dlat = lat2 - lat1
  a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  c = 2 * atan2(sqrt(a), sqrt(1 - a))
  distance = EARTH_RADIUS * c
  return distance

def get_best_value(best):
  return float(best.split('|')[1])

def find_aq(coords):
  coords = coords.replace('}', '').replace('{', '').split(',')
  lat = coords[0]
  lon = coords[1]
  best = "0|99999"
  for index, row in unique_aq.iterrows():
    aq_coords = row["coords"]
    lataq = get_lat(aq_coords)
    lonaq = get_lon(aq_coords)
    distance = get_distance(lat, lon, lataq, lonaq)
    #if(float(distance) <=  2):
    #  return f'{row["aq_station_name"]}|{distance}'
    if (float(distance) < get_best_value(best)):
      best = f'{row["aq_station_name"]}|{distance}'
  return best
def get_air_distance(val):
  return val.split('|')[1]
def get_air_station_name(val):
  return val.split('|')[0]
unique_coords['air-quality-values'] = unique_coords.coords.apply(lambda x: find_aq(x))
unique_coords['air-quality-station'] = unique_coords["air-quality-values"].apply(lambda x: get_air_station_name(x))
unique_coords['air-quality-distance'] = unique_coords["air-quality-values"].apply(lambda x: get_air_distance(x))
unique_coords.head()

In [ ]:
unique_coords = unique_coords[['coords', 'air-quality-station', 'air-quality-distance']]
unique_coords['air-quality-station'].value_counts()

In [ ]:
df_bike_aqstation = df_bike.merge(how='left', right=unique_coords, left_on='startcoords', right_on='coords').drop('startcoords', axis=1).drop('coords', axis=1).fillna("-")
df_bike_aqstation.head()

In [ ]:
df_aq_o3 = df_aq[df_aq['parameter'] == 'o3']
df_aq_co = df_aq[df_aq['parameter'] == 'co']
df_aq_o3 = df_aq_o3[['date',	'parameter',	'location',	'value']]
df_aq_co = df_aq_co[['date',	'parameter',	'location',	'value']]
df_aq_co

In [ ]:
df_bike_aqstation.columns = ['tripduration', 'starttime', 'start station id',
       'start station latitude', 'start station longitude', 'end station id',
       'end station latitude', 'end station longitude', 'aqstation',
       'aqdistance']
df_bike_aqstation.head()

In [ ]:
df_bike_aqstation = df_bike_aqstation.merge(right=df_aq_o3, how='left', left_on=['starttime','aqstation'], right_on=['date','location']).drop(['date', 'location', 'parameter'], axis=1).fillna("-")
df_bike_aqstation.columns = ['tripduration',	'starttime',	'start station id',	'start station latitude',	'start station longitude',	'end station id',	'end station latitude',	'end station longitude', 'aqstation', 'aqdistance', 'o3']
df_bike_aqstation.head()

In [ ]:
df_bike_aqstation = df_bike_aqstation.merge(right=df_aq_co, how='left', left_on=['starttime','aqstation'], right_on=['date','location']).drop(['date', 'location', 'parameter'], axis=1).fillna("-")
df_bike_aqstation.columns = ['tripduration',	'starttime',	'start station id',	'start station latitude',	'start station longitude',	'end station id',	'end station latitude',	'end station longitude', 'aqstation', 'aqdistance', 'o3', 'co']
df_bike_aqstation.head()

In [ ]:
df_bike_aqstation.columns = ['tripduration', 'starttime', 'startstationid',
       'startstationlatitude', 'startstationlongitude', 'endstationid',
       'endstationlatitude', 'endstationlongitude', 'aqstation',
       'aqdistance', 'o3', 'co']
df_bike_aqstation.head()

In [ ]:
#df_bike_aqstation.to_csv(r'gdrive/My Drive/EDA NY/csv/result/2019'+month+'.csv', index = False, header=True)

In [ ]:
'''def join_coords(lat, lon, lat2, lon2):
  return f'{lat},{lon}|{lat2},{lon2}'
df_bike_aqstation['coords'] = df_bike_aqstation.apply(lambda row: join_coords(row["startstationlatitude"], row["startstationlongitude"], row["endstationlatitude"], row["endstationlongitude"]), axis = 1)
def calculate_distance(coords):
  coords = coords.split('|')
  coords1 = coords[0].split(',')
  coords2 = coords[1].split(',')
  return get_distance_map_api(coords1[0],coords1[1],coords2[0],coords2[1])
distance_coords = df_bike_aqstation.coords.value_counts().rename_axis('coords').reset_index(name='counts')
distance_coords['distance'] = distance_coords.coords.apply(lambda x: calculate_distance(x))
distance_coords.head()'''